In [1]:
import numpy as np
from scipy import stats
from sklearn import clone
from sklearn.metrics import pairwise_distances_argmin_min

from skactiveml.base import SingleAnnotatorPoolQueryStrategy, SkactivemlClassifier
from skactiveml.utils import (
    MISSING_LABEL,
    check_type,
    check_equal_missing_label,
    unlabeled_indices
)

from sklearn.datasets import make_classification
from skactiveml.classifier import SklearnClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
class Badge(SingleAnnotatorPoolQueryStrategy):
    def __init__(
            self,
            missing_label=MISSING_LABEL,
            random_state=None
    ):
        super().__init__(
            missing_label=missing_label, random_state=random_state
        )

    def query(
            self,
            X,
            y,
            clf,
            candidates=None,
            batch_size=1,
            return_utilities=False,
            return_embeddings=False,
    ):
        # Validate input parameters
        X, y, candidates, batch_size, return_utilities = self._validate_data(
            X, y, candidates, batch_size, return_utilities, reset=True
        )

        X_cand, mapping = self._transform_candidates(candidates, X, y)

        # Validate classifier type
        check_type(clf, "clf", SkactivemlClassifier)
        check_equal_missing_label(clf.missing_label, self.missing_label_)
        if not isinstance(return_embeddings, bool):
            raise TypeError("'return_embeddings' must be a boolean.")

        # Fit the classifier
        clf = clone(clf).fit(X, y)

        # find the unlabeled dataset
        if candidates is None:
            X_unlbld = X_cand
            unlbld_mapping = mapping
        elif mapping is not None:
            unlbld_mapping = unlabeled_indices(y[mapping], missing_label=self.missing_label)
            X_unlbld = X_cand[unlbld_mapping]
            unlbld_mapping = mapping[unlbld_mapping]
        else:
            X_unlbld = X_cand
            unlbld_mapping = np.arange(len(X_cand))

        # Gradient embedding, aka predict class membership probabilities
        
        probas = clf.predict_proba(X_unlbld)
        print(probas)
        p_max = np.max(probas, axis=1).reshape(-1,1) #gaile
        g_x = (p_max - 1) * X_unlbld

        # init the utilities
        if mapping is not None:
            utilities = np.full(shape=(batch_size, X.shape[0]), fill_value=np.nan)
        else:
            utilities = np.full(shape=(batch_size, X_cand.shape[0]), fill_value=np.nan)

        # 2. sampling with kmeans++
        query_indicies = np.array([], dtype=int)
        D_p = []
        for i in range(batch_size):
            if i == 0:
                d_probas, is_randomized = _d_probability(g_x, [])
            else:
                d_probas, is_randomized = _d_probability(g_x, [idx_in_unlbld], D_p[i-1])
            D_p.append(d_probas)
            
            utilities[i, unlbld_mapping] = d_probas
            utilities[i, query_indicies] = np.nan
            
            if is_randomized:
                idx_in_unlbld = self.random_state_.choice(len(g_x))
            else:
                customDist = stats.rv_discrete(name="customDist", values=(np.arange(len(d_probas)), d_probas)) # gaile
                idx_in_unlbld_array = customDist.rvs(size=1, random_state=self.random_state_)
                idx_in_unlbld = idx_in_unlbld_array[0]
            
            idx = unlbld_mapping[idx_in_unlbld]
            query_indicies = np.append(query_indicies, [idx])
        
        if return_utilities:
            return query_indicies, utilities
        else:
            return query_indicies


def _d_probability(g_x, query_indicies, d_latest=None):
    if len(query_indicies) == 0:
        return np.ones(shape=len(g_x)), True
    g_query_indicies = g_x[query_indicies]
    _, D = pairwise_distances_argmin_min(X=g_x, Y=g_query_indicies)
    if d_latest is not None:
        print("d_latest:", d_latest)
        D = np.minimum(d_latest, D)
    D2 = np.square(D)
    print("D2 ", D2)
    D2_sum = np.sum(D2)
    if D2_sum == 0:
        return np.ones(shape=len(g_x)), True
    D_probas = D2 / D2_sum
    return D_probas, False

In [3]:
X, y_true = make_classification(n_features=2, n_redundant=0, random_state=0)
y = np.full(shape=y_true.shape, fill_value=MISSING_LABEL)
clf = SklearnClassifier(LogisticRegression(), classes=np.unique(y_true))
qs = Badge(random_state=0)

In [4]:
query = qs.query(X[:5], y[:5], clf, candidates=None, batch_size=2)
print(query)

[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]
d_latest: [1. 1. 1. 1. 1.]
D2  [1. 1. 1. 1. 0.]
[4 2]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/skactiveml/classifier/_wrapper.py:320: UserWarning: The 'base_estimator' could not be fitted because of 'There is no labeled data.'. Therefore, the class labels of the samples are counted and will be used to make predictions. The class label distribution is `_label_counts=[0, 0]`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/skactiveml/classifier/_wrapper.py:219: UserWarning: Since the 'base_estimator' could not be fitted when calling the `fit` method, the class label distribution`_label_counts=[0, 0]` is used to make the predictions.
  warnings.warn(


In [5]:
query, utilities = qs.query(X[:5], y[:5], clf, candidates=None, batch_size=3, return_utilities=True)
print(query)
print(utilities)

[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]
d_latest: [1. 1. 1. 1. 1.]
D2  [1. 1. 1. 1. 0.]
d_latest: [0.25 0.25 0.25 0.25 0.  ]
D2  [0.0625    0.0126093 0.        0.0625    0.       ]
[4 2 3]
[[1.         1.         1.         1.         1.        ]
 [0.25       0.25       0.25       0.25              nan]
 [0.45418441 0.09163119        nan 0.45418441        nan]]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/skactiveml/classifier/_wrapper.py:320: UserWarning: The 'base_estimator' could not be fitted because of 'There is no labeled data.'. Therefore, the class labels of the samples are counted and will be used to make predictions. The class label distribution is `_label_counts=[0, 0]`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/skactiveml/classifier/_wrapper.py:219: UserWarning: Since the 'base_estimator' could not be fitted when calling the `fit` method, the class label distribution`_label_counts=[0, 0]` is used to make the predictions.
  warnings.warn(


In [6]:
y[1] = y_true[1]
y[0] = y_true[0]
query, utilities = qs.query(X[:5], y[:5], clf, candidates=None, batch_size=3, return_utilities=True)
print(query)
print(utilities)

[[1. 0.]
 [1. 0.]
 [1. 0.]]
d_latest: [1. 1. 1.]
D2  [0. 0. 0.]
d_latest: [1. 1. 1.]
D2  [0. 0. 0.]
[2 3 4]
[[nan nan  1.  1.  1.]
 [nan nan nan  1.  1.]
 [nan nan nan nan  1.]]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/skactiveml/classifier/_wrapper.py:320: UserWarning: The 'base_estimator' could not be fitted because of 'This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0'. Therefore, the class labels of the samples are counted and will be used to make predictions. The class label distribution is `_label_counts=[2, 0]`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/skactiveml/classifier/_wrapper.py:219: UserWarning: Since the 'base_estimator' could not be fitted when calling the `fit` method, the class label distribution`_label_counts=[2, 0]` is used to make the predictions.
  warnings.warn(


In [7]:
query, utilities = qs.query(X, y, clf, candidates=np.arange(4,10), batch_size=3, return_utilities=True)
print(query)
print(utilities)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
d_latest: [1. 1. 1. 1. 1. 1.]
D2  [0. 0. 0. 0. 0. 0.]
d_latest: [1. 1. 1. 1. 1. 1.]
D2  [0. 0. 0. 0. 0. 0.]
[7 5 9]
[[nan nan nan nan  1.  1.  1.  1.  1.  1. nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan  1.  1.  1. nan  1.  1. nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan]
 [n

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/skactiveml/classifier/_wrapper.py:320: UserWarning: The 'base_estimator' could not be fitted because of 'This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0'. Therefore, the class labels of the samples are counted and will be used to make predictions. The class label distribution is `_label_counts=[2, 0]`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/skactiveml/classifier/_wrapper.py:219: UserWarning: Since the 'base_estimator' could not be fitted when calling the `fit` method, the class label distribution`_label_counts=[2, 0]` is used to make the predictions.
  warnings.warn(
